# Computer Science 2XC3 - Graded Lab II

Please refer to the pdf for detailed instructions. The below file contains all the preliminary code you will need to work on the lab. You can copy paste instructions here to create one cohesive lab and organize it that best suits your teams workflow. 

In [31]:
import random
import timeit 
import matplotlib.pyplot as plt
import numpy as np
import math
from collections import deque

In [54]:
class Graph:
    def __init__(self, nodes):
        self.graph = [[] for _ in range(nodes)]
 
    def has_edge(self, src, dst):
        return src in self.graph[dst]
    
    def add_edge(self,src,dst):
        if not self.has_edge(src,dst):
            self.graph[src].append(dst)
            self.graph[dst].append(src)
    
    def get_graph(self):
        return self.graph

# 1.5
def create_random_graph(n, num_edges):
    g = Graph(n)

    # clean input, any more edges and we'd have repeats
    max_edges = (n*(n-1))/2
    adjusted_edges = int(min(num_edges, max_edges))

    for _ in range(adjusted_edges):
        while True:
            start = random.randint(0, n-1)
            end = random.randint(0, n-1)

            if start == end or g.has_edge(start, end):
                continue
            g.add_edge(start, end)
            break

    return g

myg = create_random_graph(20, 2)
print(myg.get_graph())

[[13], [], [], [], [], [], [15], [], [], [], [], [], [], [0], [], [6], [], [], [], []]


In [59]:
# 1.1

def bfs(graph: Graph, n1, n2):
    if n1 == n2:
        return []
    g = graph.get_graph()

    # have we seen this node?
    visited = set()
    visited.add(n1)
    q = deque()
    q.extend(g[n1])
    found = False

    # construct a path for all nodes so we can backtrack to the first node
    # each key is a child and the value is the parent node
    path = {n1: n1}
    for first_children in g[n1]:
        path[first_children] = n1

    while len(q) > 0 and not found:
        # ensure we look at the oldest elements first
        n = q.popleft()

        # run checks
        if n == n2:
            found = True
        if n in visited:
            continue

        # save info of node
        visited.add(n)
        q.extend(g[n])
        for child in g[n]:
            if child not in visited:
                path[child] = n

    if not found:
        return []

    # generate our list path
    path_out = []
    n = n2
    while n != n1:
        path_out.append(n)
        n = path[n]
    path_out.append(n1)
    path_out.reverse()
    return path_out

my_bfs = bfs(myg, 1, 1)
print(my_bfs)

[]


In [ ]:
def depth_first_search(G,node,end_point=None):
    stack = [node]
    graph = G.get_graph()
    seen=set()

    while len(stack) !=0:
        node = stack.pop()
        # search for neighbours in graph
        if node not in seen:
            seen.add(node)
            print("Visited node:" + str(node))
            # if the given node has an edge
            if node in graph.keys():
                # iterate over edges of node
                for nn in graph[node]:

                    # limited traversal
                    if nn == end_point:
                        return True
                    # add to stack
                    stack.append(nn)

In [ ]:
#Breadth First Search
def breadth_first_search(G, start_node):
    stack = [start_node]
    graph = G.get_graph()
    seen=set()

    seen.add(start_node)

    while len(stack) > 0:
        node = stack[0]
        stack = stack[1:]
        print("Visiting node: " + str(node))
        if node in graph.keys():
            for nn in graph[node]:
                #if node == node2:
                #    return True
                if nn not in seen:
                    stack.append(nn)
                    seen.add(nn)

In [ ]:
#Use the methods below to determine minimum vertex covers

def add_to_each(sets, element):
    copy = sets.copy()
    for set in copy:
        set.append(element)
    return copy

def power_set(set):
    if set == []:
        return [[]]
    return power_set(set[1:]) + add_to_each(power_set(set[1:]), set[0])

def is_vertex_cover(G, C):
    for start in G.adj:
        for end in G.adj[start]:
            if not(start in C or end in C):
                return False
    return True

def MVC(G):
    nodes = [i for i in range(G.get_size())]
    subsets = power_set(nodes)
    min_cover = nodes
    for subset in subsets:
        if is_vertex_cover(G, subset):
            if len(subset) < len(min_cover):
                min_cover = subset
    return min_cover
